# DE Homework W5

In [34]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [7]:
pyspark.__file__

'/home/lars/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

### Question 1: What's the output of `spark.__version__`?

In [13]:
pyspark.__version__

'3.3.2'

In [25]:
spark = (
    SparkSession.builder
    .master("local[*]")
    .appName('test')
    .getOrCreate()
)

## Read Yellow 2024-10 data

In [16]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-11 16:26:48--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.222, 18.155.128.187, 18.155.128.46, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   215MB/s    in 0.3s    

2025-03-11 16:26:48 (215 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [77]:
df = (
    spark.read
    .option("header", "true")
    .parquet('yellow_tripdata_2024-10.parquet')
)

In [78]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

### Question 2: What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)?

In [79]:
df_partitioned = df.repartition(4)

In [80]:
df_partitioned.write.parquet('yellow_tripdata/2024/10/')

AnalysisException: path file:/home/lars/repos/data_engineering_zoomcamp/05-batch/yellow_tripdata/2024/10 already exists.

### Question 3: How many taxi trips were there on the 15th of October?

In [93]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- tpep_pickup_date: date (nullable = true)
 |-- tpep_dropoff_date: date (nullable = true)
 |-- trip_duration_in_seconds: long (null

In [82]:
df = df.withColumn("tpep_pickup_date", F.to_date(F.col("tpep_pickup_datetime")))
df = df.withColumn("tpep_dropoff_date", F.to_date(F.col("tpep_dropoff_datetime")))

In [83]:
df_filtered = (
    df.filter(
        (F.col("tpep_pickup_date") == "2024-10-15") &
        (F.col("tpep_dropoff_date") == "2024-10-15")
    )
)

In [84]:
df_filtered.select(F.count("tpep_pickup_date")).show(truncate=False)

+-----------------------+
|count(tpep_pickup_date)|
+-----------------------+
|127993                 |
+-----------------------+



### Question 4: What is the length of the longest trip in the dataset in hours?

In [85]:
df = df.withColumn("trip_duration_in_seconds", F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long"))
df = df.withColumn("trip_duration_in_hours", F.round(F.col("trip_duration_in_seconds")/3600, 4))

In [86]:
df.select(F.max("trip_duration_in_hours")).show(truncate=False)

+---------------------------+
|max(trip_duration_in_hours)|
+---------------------------+
|162.6178                   |
+---------------------------+



### Question 5: Spark’s User Interface which shows the application's dashboard runs on which local port?

Answer is port 4040

### Question 6: Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [71]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-11 16:58:42--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.46, 18.155.128.6, 18.155.128.187, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.001s  

2025-03-11 16:58:42 (7.84 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [75]:
df_zones = (
    spark.read
    .option("header", "true")
    .csv('taxi_zone_lookup.csv')
)

In [76]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [90]:
df.select("PULocationID").groupby(F.col("PULocationID")).count().orderBy("count").show()

+------------+-----+
|PULocationID|count|
+------------+-----+
|         105|    1|
|         199|    2|
|           5|    2|
|           2|    3|
|         111|    3|
|          44|    4|
|         187|    4|
|          84|    4|
|         204|    4|
|         245|    4|
|          59|    6|
|         109|    6|
|         156|    7|
|         118|    7|
|         206|    9|
|         176|    9|
|          30|   10|
|         172|   10|
|         251|   12|
|         184|   12|
+------------+-----+
only showing top 20 rows



In [96]:
df_zones.filter(F.col("LocationID") == 105).show()

+----------+---------+--------------------+------------+
|LocationID|  Borough|                Zone|service_zone|
+----------+---------+--------------------+------------+
|       105|Manhattan|Governor's Island...| Yellow Zone|
+----------+---------+--------------------+------------+

